In [13]:
import pandas as pd
import numpy as np

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')
path = "gdrive/My Drive/Machine Learning/Lab08"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
df = pd.read_csv(path + "/df_train.csv")
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(df, test_size=0.1)
print('Train data:')
display(df_train.head(5))
print('Valid data:')
display(df_valid.head(5))

Train data:


,timestamp,#followers,#friends,#favorites,hashtags,sentiment_1,sentiment_2,#retweet
3874732,Tue Mar 17 14:17:54 +0000 2020,523,817,0,coronavirus,1,-1,1
6018421,Mon Feb 10 22:35:08 +0000 2020,3970,1771,0,HandmadeGifts LabradoriteHappens,1,-1,1
247774,Wed Nov 06 23:57:43 +0000 2019,18,95,0,null;,1,-1,0
5069811,Tue Mar 24 19:45:26 +0000 2020,1036,2693,0,BREAKING coronavirus,3,-1,1
6696390,Fri Feb 21 10:51:39 +0000 2020,189,125,0,null;,1,-3,0


Valid data:


,timestamp,#followers,#friends,#favorites,hashtags,sentiment_1,sentiment_2,#retweet
5693854,Sun Oct 13 18:44:11 +0000 2019,105943,630,830,null;,1,-1,52
4527564,Mon Mar 16 22:07:16 +0000 2020,60,190,0,null;,1,-3,0
1915627,Wed Oct 09 19:48:15 +0000 2019,4897,130,0,null;,2,-1,0
5963549,Wed Feb 26 19:03:14 +0000 2020,24134,12190,48,null;,1,-1,12
3597801,Mon Jan 13 12:00:09 +0000 2020,530,473,0,null;,2,-1,1


**Feature selection**

In [16]:
import statsmodels.api as sm

In [17]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

In [18]:
column = []
for i in df_train.columns:
  if (df_train.dtypes[i] != 'object' and i != '#retweet'):
    column.append(i)
column

['#followers', '#friends', '#favorites', 'sentiment_1', 'sentiment_2']

In [19]:
forward_selection(df_train[column], df_train['#retweet'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


['#followers', '#favorites', '#friends', 'sentiment_2', 'sentiment_1']

In [20]:
X_train = df_train[column].values    
X_valid = df_valid[column].values

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_train = (X_train - X_means)/X_stds
X_valid = (X_valid - X_means)/X_stds

y_train = df_train['#retweet'].values
y_valid = df_test['#retweet'].values

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(5,))    ## tạo một layer Input, mỗi sample có input là 1 vector độ dài 5
dense_1 = Dense(units=10, activation='relu')(inputs)   ## tạo một layer gồm 10 units, hàm kích hoạt relu
outputs = Dense(units=1, activation=None)(dense_1)  
model = Model(inputs=inputs, outputs=outputs)    ## tạo model với input và output layer đã xây dựng

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')      ## khai báo optimizer và loss cho model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 71
Trainable params: 71
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size = 256)

Epoch 1/10
25792/25792 [==============================] - 30s 1ms/step - loss: 382516.3125 - val_loss: 303897.7188
Epoch 2/10
25792/25792 [==============================] - 30s 1ms/step - loss: 378727.8750 - val_loss: 303709.8438
Epoch 3/10
25792/25792 [==============================] - 30s 1ms/step - loss: 373722.7500 - val_loss: 303983.0312
Epoch 4/10
25792/25792 [==============================] - 30s 1ms/step - loss: 367371.0938 - val_loss: 304422.6562
Epoch 5/10
25792/25792 [==============================] - 30s 1ms/step - loss: 359798.8750 - val_loss: 305020.7188
Epoch 6/10
25792/25792 [==============================] - 30s 1ms/step - loss: 351648.8438 - val_loss: 305762.6250
Epoch 7/10
25792/25792 [==============================] - 30s 1ms/step - loss: 342008.6875 - val_loss: 306475.1875
Epoch 8/10
25792/25792 [==============================] - 30s 1ms/step - loss: 332056.4062 - val_loss: 307557.6250
Epoch 9/10
25792/25792 [==============================] - 31s 1ms/step - loss: 3

In [23]:
df_test = pd.read_csv(path + "/df_test.csv")
X_test = df_test[column].values
X_test = (X_test - X_means)/X_stds
y_test = df_test['#retweet'].values

In [24]:
y_test.shape

(815153,)

In [25]:
y_pred = model.predict(X_test)

In [26]:
y_pred = np.round(y_pred)
y_pred = y_pred.astype(int)
y_pred = np.squeeze(y_pred)
y_pred.shape

(815153,)

In [27]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(y_test, y_pred)

1.26032918948597